In [329]:
import yfinance as yf

import os
os.chdir('C:\\Users\\btindol\\OneDrive - Stryker\\Linked In Learn\\Algorithmic Investing\Algorithmic Trading & Quantitative Analysis using Python')  
print(os.getcwd())  # Prints the current working directory

#####################################################################################################################################################
#imports 
#!pip install yfinance 
#!pip install pandas 
#!pip install numpy
#!pip install requests
!pip install datetime
#!pip install yahoofinancials
!pip install stocktrends
from yahoofinancials import YahooFinancials

import pandas as pd
import numpy as np
import requests
import datetime as dt
import os
import json 
from functools import reduce
import statsmodels.api as sm
from stocktrends import Renko




import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\btindol\OneDrive - Stryker\Linked In Learn\Algorithmic Investing\Algorithmic Trading & Quantitative Analysis using Python

You should consider upgrading via the 'c:\users\btindol\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\btindol\anaconda3\python.exe -m pip install --upgrade pip' command.


# SINGLE STOCK ANALYZER NOW MAKE STRATEGY OFF OF IT

# CONSOLIDATION FOR EASE!!!!!!!!!!!!!!


In [348]:
# for each of the column names for the % change columns 

def shifted_column(df):
    pctchg = [col for col in df.columns if 'percent_change' in col] # grabe % change columns
    colnew = []                                                                    # initiate empty list
    for i in range(0,len(pctchg)):                                                 # loop over column names
        print(pctchg[i])
        var1 = df[pctchg[i]].shift(-1)                                            # grab colunmn by name and shift up one
        colnew.append(var1)                                                        # append to list
    colnew = pd.DataFrame(colnew)                                                  # now its wide dataframe
    colnew = colnew.transpose()                                                    # make it narrow
    cols = colnew.columns.tolist()                                                 # take column names of narrow dataframe                                                
    newcols = []                                                                   # make list to add the text shift to the column name
    for j in range(0,len(cols)): 
        colz = cols[j] + "_" + "shift"                                             # add shift to the name of shifted column

        newcols.append(colz) 
    colnew.columns = newcols  
    return colnew
    #########################################################################################
    #########################################################################################
    
def clean_up_zeros_and_column_names(df):
    df =df.fillna(0)     
    df.columns = df.columns.str.replace("-", "_") 
    return df

In [349]:
def get_current_position(df,initial_position):
    
    # Calculate returns from initial position entry 
    shiftval = [col for col in df.columns if 'shift' in col]                   # find the shift column to bring % change up to today to mutliply to get positions next day amount.. 

    initial_position = initial_position                                             # pass from function above

    new = [(df[shiftval[0]][0] * initial_position) + initial_position]        # get first value in final for iterate the 0 of this 
    for i in range(1, len(df.index)):                                          # starting at second row of the dataframe column 
        new.append((new[i-1]*df[shiftval].values[i][0]) +new[i-1])
    
    df['Position'] = new
    df['Position'] = df['Position'].shift(1)
    df['Position'][0] = initial_position
    df.fillna(0)
    return df#['Position']

In [350]:
# Calculating RSI without using loop
def rsi(df, n):
    "function to calculate RSI"
    delta = df["Adj Close"].diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[n-1]] = np.mean( u[:n]) # first value is average of gains
    u = u.drop(u.index[:(n-1)])
    d[d.index[n-1]] = np.mean( d[:n]) # first value is average of losses
    d = d.drop(d.index[:(n-1)])
    rs = u.ewm(com=n,min_periods=n).mean()/d.ewm(com=n,min_periods=n).mean()
    return 100 - 100 / (1+rs)

In [351]:
def ATR(DF,n):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Adj Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Adj Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    #df['ATR'] = df['TR'].ewm(span=n,adjust=False,min_periods=n).mean()
    df2 = df.drop(['H-L','H-PC','L-PC'],axis=1)
    df2.dropna(inplace=True)
    return df2

In [352]:
def ADX(DF,n):
    "function to calculate ADX"
    df2 = DF.copy()
    df2['TR'] = ATR(df2,n)['TR'] #the period parameter of ATR function does not matter because period does not influence TR calculation
    df2['DMplus']=np.where((df2['High']-df2['High'].shift(1))>(df2['Low'].shift(1)-df2['Low']),df2['High']-df2['High'].shift(1),0)
    df2['DMplus']=np.where(df2['DMplus']<0,0,df2['DMplus'])
    df2['DMminus']=np.where((df2['Low'].shift(1)-df2['Low'])>(df2['High']-df2['High'].shift(1)),df2['Low'].shift(1)-df2['Low'],0)
    df2['DMminus']=np.where(df2['DMminus']<0,0,df2['DMminus'])
    TRn = []
    DMplusN = []
    DMminusN = []
    TR = df2['TR'].tolist()
    DMplus = df2['DMplus'].tolist()
    DMminus = df2['DMminus'].tolist()
    for i in range(len(df2)):
        if i < n:
            TRn.append(np.NaN)
            DMplusN.append(np.NaN)
            DMminusN.append(np.NaN)
        elif i == n:
            TRn.append(df2['TR'].rolling(n).sum().tolist()[n])
            DMplusN.append(df2['DMplus'].rolling(n).sum().tolist()[n])
            DMminusN.append(df2['DMminus'].rolling(n).sum().tolist()[n])
        elif i > n:
            TRn.append(TRn[i-1] - (TRn[i-1]/n) + TR[i])
            DMplusN.append(DMplusN[i-1] - (DMplusN[i-1]/n) + DMplus[i])
            DMminusN.append(DMminusN[i-1] - (DMminusN[i-1]/n) + DMminus[i])
    df2['TRn'] = np.array(TRn)
    df2['DMplusN'] = np.array(DMplusN)
    df2['DMminusN'] = np.array(DMminusN)
    df2['DIplusN']=100*(df2['DMplusN']/df2['TRn'])
    df2['DIminusN']=100*(df2['DMminusN']/df2['TRn'])
    df2['DIdiff']=abs(df2['DIplusN']-df2['DIminusN'])
    df2['DIsum']=df2['DIplusN']+df2['DIminusN']
    df2['DX']=100*(df2['DIdiff']/df2['DIsum'])
    ADX = []
    DX = df2['DX'].tolist()
    for j in range(len(df2)):
        if j < 2*n-1:
            ADX.append(np.NaN)
        elif j == 2*n-1:
            ADX.append(df2['DX'][j-n+1:j+1].mean())
        elif j > 2*n-1:
            ADX.append(((n-1)*ADX[j-1] + DX[j])/n)
    df2['ADX']=np.array(ADX)
    return df2['ADX']

In [353]:
def slope(ser,n):
    "function to calculate the slope of regression line for n consecutive points on a plot"
    ser = (ser - ser.min())/(ser.max() - ser.min())
    x = np.array(range(len(ser)))
    x = (x - x.min())/(x.max() - x.min())
    slopes = [i*0 for i in range(n-1)]
    for i in range(n,len(ser)+1):
        y_scaled = ser[i-n:i]
        x_scaled = x[:n]
        x_scaled = sm.add_constant(x_scaled)
        model = sm.OLS(y_scaled,x_scaled)
        results = model.fit()
        slopes.append(results.params[-1])
    slope_angle = (np.rad2deg(np.arctan(np.array(slopes))))
    return np.array(slope_angle)



# Come back for Renko Later!!


In [354]:
# def renko_DF(DF):
#     "function to convert ohlc data into renko bricks"
#     df = DF.copy()
#     df.reset_index(inplace=True)
#     df = df.loc[:,["Date","Open","High","Low","Adj Close","Volume"]]
#     df.rename(columns = {"Date" : "date", "High" : "high","Low" : "low", "Open" : "open","Adj Close" : "close", "Volume" : "volume"}, inplace = True)
#     df2 = Renko(df)
#     df2.brick_size = round(ATR(DF,120)["ATR"][-1],0)
#     renko_df = df2.get_ohlc_data() #if using older version of the library please use get_bricks() instead
#     return renko_df


# def renko_DF(DF):
#     "function to convert ohlc data into renko bricks"
#     df = DF.copy()
#     df.reset_index(inplace=True)
#     df = df.loc[:,["Date","Open","High","Low","Close","Volume"]]
#     df.columns = ["date","open","high","low","close","volume"]
#     df2 = Renko(df)
#     df2.brick_size = max(0.5,round(ATR(DF,120),0))
#     renko_df = df2.get_bricks()
#     renko_df["bar_num"] = np.where(renko_df["uptrend"]==True,1,np.where(renko_df["uptrend"]==False,-1,0))
#     for i in range(1,len(renko_df["bar_num"])):
#         if renko_df["bar_num"][i]>0 and renko_df["bar_num"][i-1]>0:
#             renko_df["bar_num"][i]+=renko_df["bar_num"][i-1]
#         elif renko_df["bar_num"][i]<0 and renko_df["bar_num"][i-1]<0:
#             renko_df["bar_num"][i]+=renko_df["bar_num"][i-1]
#     renko_df.drop_duplicates(subset="date",keep="last",inplace=True)
#     return renko_df

In [355]:
def OBV(DF):
    """function to calculate On Balance Volume"""
    df = DF.copy()
    df['daily_ret'] = df['Adj Close'].pct_change()
    df['direction'] = np.where(df['daily_ret']>=0,1,-1)
    df['direction'][0] = 0
    df['vol_adj'] = df['Volume'] * df['direction']
    df['obv'] = df['vol_adj'].cumsum()
    return df['obv']


In [356]:
def single_stock_analyzer(stocks, begin_date,end_date,initial_position,slow_ma,fast_ma):

    # looping over tickers and creating a dataframe with closing prices
    # get just adjusted close column for that ticker
    for ticker in stocks:
        AdjclosePrice = yf.download(ticker,beg_date,end_date)[['Adj Close','Open','High','Low','Close','Volume']]     
    
    # Make it a dataframe
    AdjclosePrice = pd.DataFrame(AdjclosePrice)
    
    # pass dataframe into function that gets the adjusted close percent change column
    AdjclosePrice_PctChg = get_daily_returns(AdjclosePrice)
    
    # merge percent change column to normal price of the stock (Adj Close)
    final = AdjclosePrice.merge(AdjclosePrice_PctChg, on='Date', how='left')
    
    # Shift the percent change column up for multiplying by the initial position input column later to simulate earnings.. 
    colnew =shifted_column(AdjclosePrice_PctChg)

    # join the new column onto existing Adj Close and Adj Close percent change columns 
    final_f= final.merge(colnew, on='Date', how='left')

    # fill na's with 0's and change weird characters such as - to _ in column names 
    final_f = clean_up_zeros_and_column_names(final_f)

    # See how the money you put in performed.. 
    var = get_current_position(final_f,initial_position)
    final_f['Position'] = var
    
    # Get the moving averages for the crossover strategt slow and fast moving averages..  
    final_f['Fast_moving_average'] = final_f['Adj Close'].rolling(window=fast_ma).mean() # 5 day price moving average
    final_f['Slow_moving_average'] = final_f['Adj Close'].rolling(window=slow_ma).mean() # 5 day price moving average

    # Bolligner band of the fast moving average 30 day.. 
    final_f["BB_up"] = final_f["Fast_moving_average"] + 2*final_f['Adj Close'].rolling(fast_ma).std(ddof=0) #ddof=0 is required since we want to take the standard deviation of the population and not sample
    final_f["BB_dn"] = final_f["Fast_moving_average"] - 2*final_f['Adj Close'].rolling(fast_ma).std(ddof=0) #ddof=0 is required since we want to take the standard deviation of the population and not sample
    final_f["BB_width"] = final_f["BB_up"] - final_f["BB_dn"]
    
    # rsi standard 14 periods
    final_f["RSI_fast_14_periods"] = rsi(final_f,14) # standard periods is 14
    final_f["RSI_slow_30_periods"] = rsi(final_f,30) # longer than average periods 30 
    
    # Average true range 14 period is standard 
    final_f2 = ATR(final_f,14) # 14 is standard for longer use 20 
    
    # Average directional movement here we will use a period matching the fast moving average
    final_f2['ADX'] = ADX(final_f2,fast_ma)
    
    
    #MACD, short for moving average convergence/divergence    
    """function to calculate MACD
       typical values a = 12; b =26, c =9"""
    a = 12;b=26;c=9
    final_f2["MA_Fast"]=final_f2["Adj Close"].ewm(span=a,min_periods=a).mean()
    final_f2["MA_Slow"]=final_f2["Adj Close"].ewm(span=b,min_periods=b).mean()
    final_f2["MACD"]=final_f2["MA_Fast"]-final_f2["MA_Slow"]
    final_f2["Signal"]=final_f2["MACD"].ewm(span=c,min_periods=c).mean()
    
    
    # Slope for last 5 days change if needed
    final_f2["close_slope"] = slope(final_f2["Adj Close"],5)
    
    # OBV
    OBV2= OBV(final_f2)
    
    # Merge OBV 
    final_f3 = final_f2.merge(OBV2, on='Date', how='left')
    
    return final_f3

In [388]:
final_f = single_stock_analyzer(stocks,beg_date,end_date,10.239700,30,90)
final_f

[*********************100%***********************]  1 of 1 completed
Adj Close_percent_change
Open_percent_change
High_percent_change
Low_percent_change
Close_percent_change
Volume_percent_change


<ipython-input-349-d7cc4537d8de>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Position'][0] = initial_position
<ipython-input-355-444514be0a6c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['direction'][0] = 0


,Adj Close,Open,High,Low,Close,Volume,Adj Close_percent_change,Open_percent_change,High_percent_change,Low_percent_change,...,RSI_slow_30_periods,TR,ATR,ADX,MA_Fast,MA_Slow,MACD,Signal,close_slope,obv
Date,,,,,,,,,,,,,,,,,,,,,
2016-08-11,11.690900,12.138200,12.272400,11.633800,11.690900,17505000,-0.037247,-0.007490,-0.014882,-0.029101,...,46.994302,0.638600,0.900431,NaN,NaN,NaN,NaN,NaN,0.000000,0
2016-08-12,11.778500,11.688900,12.041600,11.635100,11.778500,13690100,0.007493,-0.037015,-0.018806,0.000112,...,47.258066,0.406500,0.906867,NaN,NaN,NaN,NaN,NaN,0.000000,13690100
2016-08-13,11.571300,11.770400,11.791100,11.511800,11.571300,7539490,-0.017591,0.006972,-0.020803,-0.010597,...,46.690200,0.279300,0.902510,NaN,NaN,NaN,NaN,NaN,0.000000,6150610
2016-08-14,11.194800,11.583300,11.694600,11.099100,11.194800,8680160,-0.032537,-0.015896,-0.008184,-0.035850,...,45.660002,0.595500,0.876289,NaN,NaN,NaN,NaN,NaN,0.000000,-2529550
2016-08-15,11.218400,11.201700,11.310800,11.000100,11.218400,7500840,0.002108,-0.032944,-0.032819,-0.008920,...,45.737550,0.310699,0.821903,NaN,NaN,NaN,NaN,NaN,-3.640140,4971290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-09,3928.844727,3911.463135,3981.259033,3743.989014,3928.844727,50568290278,0.006713,0.123342,0.007872,0.084030,...,74.809571,237.270020,246.107265,NaN,3333.180026,2925.372969,407.807056,310.724455,88.875224,1599494182043
2021-05-10,3952.293945,3924.413330,4197.473145,3684.451660,3952.293945,62691789007,0.005968,0.003311,0.054308,-0.015902,...,75.004114,513.021484,266.464355,NaN,3428.428321,3001.441190,426.987131,333.976991,88.992908,1662185971050
2021-05-11,4168.701172,3948.271973,4178.208984,3783.889404,4168.701172,52679737865,0.054755,0.006080,-0.004589,0.026988,...,76.718741,394.319580,280.985910,NaN,3542.316452,3087.904892,454.411560,358.063904,89.028882,1714865708915


# LEFT OFF HERE !!! TRYING TO IMPLEMENT STRATEGY!!

# Get crossover strategy here then get multiple crossovers.. 

In [389]:
short_window= 30
long_window = 90
CrossOver_Strat = final_f[["Adj Close"]]

# Short and long moving average.. 
CrossOver_Strat["short_mavg"] = CrossOver_Strat["Adj Close"].rolling(window= short_window,min_periods=1, center=False).mean()
CrossOver_Strat["long_mavg"] = CrossOver_Strat["Adj Close"].rolling(window= long_window,min_periods=1, center=False).mean()

CrossOver_Strat["signal"]=0
CrossOver_Strat["signal"][short_window:] =np.where(CrossOver_Strat['short_mavg'][short_window:] > CrossOver_Strat['long_mavg'][short_window:],1,0)

CrossOver_Strat["positions"] = CrossOver_Strat["signal"].diff()

CrossOver_Strat['buy'] = np.where(CrossOver_Strat['signal'] == 1,1000,0) #
CrossOver_Strat

<ipython-input-389-1901ec1ba91c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CrossOver_Strat["short_mavg"] = CrossOver_Strat["Adj Close"].rolling(window= short_window,min_periods=1, center=False).mean()
<ipython-input-389-1901ec1ba91c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CrossOver_Strat["long_mavg"] = CrossOver_Strat["Adj Close"].rolling(window= long_window,min_periods=1, center=False).mean()
<ipython-input-389-1901ec1ba91c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy

,Adj Close,short_mavg,long_mavg,signal,positions,buy
Date,,,,,,
2016-08-11,11.690900,11.690900,11.690900,0,NaN,0
2016-08-12,11.778500,11.734700,11.734700,0,0.0,0
2016-08-13,11.571300,11.680233,11.680233,0,0.0,0
2016-08-14,11.194800,11.558875,11.558875,0,0.0,0
2016-08-15,11.218400,11.490780,11.490780,0,0.0,0
...,...,...,...,...,...,...
2021-05-09,3928.844727,2708.088615,2093.142912,1,0.0,1000
2021-05-10,3952.293945,2768.633675,2117.412455,1,0.0,1000
2021-05-11,4168.701172,2835.668481,2144.350875,1,0.0,1000


In [390]:
# Set the initial capital to 100k

final_dfz = final_f.merge(CrossOver_Strat,on='Date', how='left')
final_dfz

,Adj Close_x,Open,High,Low,Close,Volume,Adj Close_percent_change,Open_percent_change,High_percent_change,Low_percent_change,...,MACD,Signal,close_slope,obv,Adj Close_y,short_mavg,long_mavg,signal,positions,buy
Date,,,,,,,,,,,,,,,,,,,,,
2016-08-11,11.690900,12.138200,12.272400,11.633800,11.690900,17505000,-0.037247,-0.007490,-0.014882,-0.029101,...,NaN,NaN,0.000000,0,11.690900,11.690900,11.690900,0,NaN,0
2016-08-12,11.778500,11.688900,12.041600,11.635100,11.778500,13690100,0.007493,-0.037015,-0.018806,0.000112,...,NaN,NaN,0.000000,13690100,11.778500,11.734700,11.734700,0,0.0,0
2016-08-13,11.571300,11.770400,11.791100,11.511800,11.571300,7539490,-0.017591,0.006972,-0.020803,-0.010597,...,NaN,NaN,0.000000,6150610,11.571300,11.680233,11.680233,0,0.0,0
2016-08-14,11.194800,11.583300,11.694600,11.099100,11.194800,8680160,-0.032537,-0.015896,-0.008184,-0.035850,...,NaN,NaN,0.000000,-2529550,11.194800,11.558875,11.558875,0,0.0,0
2016-08-15,11.218400,11.201700,11.310800,11.000100,11.218400,7500840,0.002108,-0.032944,-0.032819,-0.008920,...,NaN,NaN,-3.640140,4971290,11.218400,11.490780,11.490780,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-09,3928.844727,3911.463135,3981.259033,3743.989014,3928.844727,50568290278,0.006713,0.123342,0.007872,0.084030,...,407.807056,310.724455,88.875224,1599494182043,3928.844727,2708.088615,2093.142912,1,0.0,1000
2021-05-10,3952.293945,3924.413330,4197.473145,3684.451660,3952.293945,62691789007,0.005968,0.003311,0.054308,-0.015902,...,426.987131,333.976991,88.992908,1662185971050,3952.293945,2768.633675,2117.412455,1,0.0,1000
2021-05-11,4168.701172,3948.271973,4178.208984,3783.889404,4168.701172,52679737865,0.054755,0.006080,-0.004589,0.026988,...,454.411560,358.063904,89.028882,1714865708915,4168.701172,2835.668481,2144.350875,1,0.0,1000


In [ ]:
# Normal position without using the strategy.. 
initial_position = 100000
new = [(CrossOver_Strat.percent_change_shift.values[0] * initial_position) + initial_position] # make the first value of new column hold it in variable where we take initial position and multiply by the shifted percent change column to get initial starting value
for i in range(1, len(CrossOver_Strat.index)): # starting at second row of the dataframe column 
    new.append((new[i-1]*CrossOver_Strat.percent_change_shift.values[i]) +new[i-1]) # take new and multiply the first calculated value by the next percent change plus the origional value to get new position value

# Assign the list new to the dataframe column
CrossOver_Strat['CurrentPosition'] = new
CrossOver_Strat['CurrentPosition'] = CrossOver_Strat['CurrentPosition'].round(2) # round the coumn
CrossOver_Strat['Current_Position_Final'] = CrossOver_Strat['CurrentPosition'].round(2).shift(1) # shift it down to make since since shifted percent change up one value
CrossOver_Strat.Current_Position_Final.iloc[0] = 10000  # inject the initial starting amount into the first value of the column.. 
CrossOver_Strat

In [ ]:
var = CrossOver_Strat[CrossOver_Strat.positions == 1].first_valid_index()
var

In [ ]:
# Make new column if index is the first time ther is a cross over then add the value in..

#MAKE SURE TO SHIFT THE FINAL OUTCOME COLUMN BY THE RIGHT AMOUNT 

initial_position = 100000
var = CrossOver_Strat[CrossOver_Strat.positions == 1].first_valid_index()
var #'2016-10-27'
CrossOver_Strat["Position_Start"] = np.where(CrossOver_Strat.index == var,initial_position,0)
CrossOver_Strat["Position_plus_buy"] = CrossOver_Strat["buy"] + CrossOver_Strat["Position_Start"]
CrossOver_Strat["Current_Position_Base"] = CrossOver_Strat["Position_plus_buy"].cumsum()
CrossOver_Strat["Current_Position_Pct_Change"] =np.where(CrossOver_Strat['signal'] == 1,CrossOver_Strat["Current_Position_Base"] * CrossOver_Strat["percent_change_shift"],0)
CrossOver_Strat['Current_Position_Final2'] =CrossOver_Strat["Current_Position_Base"] + CrossOver_Strat["Current_Position_Pct_Change"]
CrossOver_Strat['Current_Position_Final3'] =np.where(CrossOver_Strat["signal"] == 0,0,CrossOver_Strat['Current_Position_Final2'])


# NEED TO FIND A WAY TO FIND O8T 
#CrossOver_Strat['cumsum_buy'] = CrossOver_Strat["buy"].cumsum()
#CrossOver_Strat["Current_Position_Pct_Change_cumbuy"] =np.where(CrossOver_Strat['signal'] == 1,CrossOver_Strat["Current_Position_Base"] * CrossOver_Strat["percent_change_shift"],0)
CrossOver_Strat.head(50)